In [1]:
from nanomesh.generator import Generator
import SimpleITK as sitk
from nanomesh.utils import show_slice, show_volume, generate_mesh_from_binary_image
import numpy as np
import math

In [2]:
gen = Generator(680, 680*math.sqrt(2), 0.24*680)

# Possible rotation/transformation of the coordinate system
theta = math.pi * 1/180
c = math.cos(theta)
s = math.sin(theta)
trans = np.array([
    [ c, 0, s],
    [ 0, 1, 0],
    [-s, 0, c]
])

vol = gen.generate_vect([100]*3, [10]*3, transform=trans, bin_val=[0.,1.])

# Convert to ITK image

im = sitk.GetImageFromArray(vol.astype('uint8'))
show_slice(im, dim='z',scale=2)

interactive(children=(IntSlider(value=49, description='z', max=99), Output()), _dom_classes=('widget-interact'…

In [5]:
im = sitk.GetImageFromArray(vol[:,:68,:].astype('uint8'))
show_slice(im, dim='z',scale=2)

interactive(children=(IntSlider(value=49, description='z', max=99), Output()), _dom_classes=('widget-interact'…

In [6]:
import pygalmesh
mesh = generate_mesh_from_binary_image(im, h=[2.5]*3, perturb=False, 
                                       lloyd=False,
                                       odt=False,
                                       max_radius_surface_delaunay_ball=0.0,
                                       max_cell_circumradius=0.,
                                       max_facet_distance=1.,
                                       exude=True,
                                       min_facet_angle=0.,
                                       max_edge_size_at_feature_edges=1.5,
                                       max_circumradius_edge_ratio=0.,
                                       bounding_box=True)
import pyvista as pv
pv.plot_itk(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [7]:
len(mesh.points)

13011

In [8]:
import pygalmesh
class Implicit3D(pygalmesh.DomainBase):
    def __init__(self):
        super().__init__()
        
        self.gen = Generator(680, math.sqrt(2)*680, 0.24*680)

        # Possible rotation/transformation of the coordinate system
        theta = math.pi * 1/180
        c = math.cos(theta)
        s = math.sin(theta)
        trans = np.array([
            [ c, 0, s],
            [ 0, 1, 0],
            [-s, 0, c]
        ])
        
        self.size = [100]*3
        self.res = [10]*3
        self.data = self.gen.generate(self.size, self.res, transform=trans, bin_val=[0.,1.])
        self.data = self.data[:,:68,:]
        
        
        self.x = np.linspace(0,1.,self.size[0])
        self.y = np.linspace(0,1.,68)
        self.z = np.linspace(0,1.,self.size[2])
        
    def eval(self, x):
        ix = np.argmin(np.abs(x[0]-self.x))
        iy = np.argmin(np.abs(x[1]-self.y))
        iz = np.argmin(np.abs(x[2]-self.z))
        if self.data[ix,iy,iz] == 1:
            return -1.
        else:
            return 0.
        
mesh = pygalmesh.generate_periodic_mesh(
    Implicit3D(),
    [0,0,0,1.,1.,1.],
    max_cell_circumradius=0.05,
    min_facet_angle=30,
    max_radius_surface_delaunay_ball=0.05,
    max_facet_distance=0.025,
    max_circumradius_edge_ratio=2.0,
    number_of_copies_in_output=1,
    # odt=True,
    # lloyd=True,
    verbose=False,
    )

In [9]:
import pyvista as pv
pv.plot_itk(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…